<a href="https://colab.research.google.com/github/paruldiwakar/ML-DL/blob/master/Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from music21 import converter,instrument,note,chord,stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils

Using TensorFlow backend.


## Read a Midi File

In [0]:
midi = converter.parse("/content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/Eternal_Harvest.mid")

In [4]:
midi

<music21.stream.Score 0x7f0b3a916d68>

In [5]:
midi.show('midi')

In [6]:
midi.show('text')

{0.0} <music21.stream.Part 0x7f0b3a88e0b8>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.tempo.MetronomeMark Quarter=240.0>
    {0.0} <music21.key.Key of E- major>
    {0.0} <music21.meter.TimeSignature 12/8>
    {0.0} <music21.stream.Voice 0x7f0b3ae17630>
        {0.0} <music21.note.Note C>
        {0.5} <music21.note.Note C>
        {1.0} <music21.note.Note C>
        {1.5} <music21.note.Note G>
        {2.0} <music21.note.Note G>
        {2.5} <music21.note.Note G>
        {3.0} <music21.note.Note B->
        {3.5} <music21.note.Note B->
        {4.0} <music21.note.Note B->
        {4.5} <music21.note.Note F>
        {5.0} <music21.note.Note F>
        {5.5} <music21.note.Note F>
        {6.0} <music21.note.Note G#>
        {6.5} <music21.note.Note G#>
        {7.0} <music21.note.Note G#>
        {7.5} <music21.note.Note E->
        {8.0} <music21.note.Note E->
        {8.5} <music21.note.Note E->
        {9.0} <music21.note

In [0]:
elements_to_parse = midi.flat.notes

In [0]:
notes_demo = []

for ele in elements_to_parse:
    
    # If the element is a note, then store its pitch
    if isinstance(ele,note.Note):
        notes_demo.append(str(ele.pitch))
    
    # If the element is a chord,split each note of chord and join them with a '+'
    elif isinstance(ele,chord.Chord):
        notes_demo.append("+".join(str(n) for n in ele.normalOrder)) 
    

In [0]:
notes_demo

In [10]:
len(notes_demo)

1123

## Preprocessing all Files

In [11]:
notes = []
folder_name = "/content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/*.mid"

for file in glob.glob(folder_name):
    midi = converter.parse(file)
    
    print("parsing %s"%file)
    
    elements_to_parse = midi.flat.notes
    
    for ele in elements_to_parse:
    
        # If the element is a note, then store its pitch
        if isinstance(ele,note.Note):
            notes.append(str(ele.pitch))

        # If the element is a chord,split each note of chord and join them with a '+'
        elif isinstance(ele,chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder)) 
    
     

parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/dontbeafraid.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/fortresscondor.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/ahead_on_our_way_piano.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/FF6epitaph_piano.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/goldsaucer.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/Ff7-One_Winged.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/Still_Alive-1.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/waltz_de_choco.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_songs/HighwindTakestotheSkies.mid
parsing /content/drive/My Drive/Colab Notebooks/midi_songs/midi_

In [12]:
len(notes)

60498

In [0]:
with open("notes","wb") as file_path:
    pickle.dump(notes,file_path)

In [0]:
with open("notes","rb") as f:
    notes = pickle.load(f)

In [15]:
n_vocab = len(set(notes))
print("Total note : ",len(notes))
print("Unique notes : ",n_vocab)

Total note :  60498
Unique notes :  359


In [16]:
print(notes[:150])

['E5', 'E4', 'G4', 'C5', 'B4', 'B3', 'F#4', 'A4', 'G4', 'G3', 'C4', 'F#4', 'E4', 'E3', 'G3', 'C4', '7+11', 'E4', 'E3', 'G3', 'A4', 'C4', 'G4', 'B3', 'B2', 'F#3', 'F#4', 'A3', 'E4', 'G3', 'G2', 'C3', 'F#3', 'E3', '11+4', '11+4', '6', '11+4', '11+4', '3', '10+3', 'F#2', 'B-2', '3', 'E-2', 'E-2', 'E-2', 'E-2', 'E-2', '3', 'E-2', '10+3', '5', '3', '10+3', '2', '9+2', 'E5', 'E3', 'E4', 'G4', 'C5', 'B4', 'B3', 'F#4', 'A4', 'G4', 'G3', 'C4', 'F#4', 'E4', 'E3', 'G3', 'C4', 'E2', 'B2', 'E4', 'E3', 'G4', 'F#4', '9+2', 'D2', 'A2', 'F#3', 'A2', 'D3', 'A2', 'C2', 'G2', 'A4', 'C3', 'G4', 'A4', 'B4', '11+4', 'E2', 'B2', 'G3', 'B2', 'E3', 'B2', 'G2', 'D3', '7+11', 'G3', 'D3', '9+1', 'A2', 'E3', 'A4', 'A3', 'E3', '11+3+6', 'B2', 'F#3', 'B3', 'F#3', 'B3', 'F#3', 'F#4', 'B3', 'F#3', '9+0+4', 'A2', 'E3', 'A4', 'A3', 'E3', 'G4', 'A3', 'E3', 'A4', 'A3', 'B4', 'E3', '11+4', 'E2', 'B2', 'E3', 'B2', 'E3', 'B2', 'E3', 'B2', '11+4', '11+4', '1+6', '6+11', '7+11+2', '2+7']


## Preparing Sequential Data for LSTM

In [0]:
# How many elements should LSTM input consider
sequence_length = 100

In [0]:
pitchnames = sorted(set(notes))

In [0]:
#Mapping between element to int value

ele_to_int = dict((ele,num) for num,ele in enumerate(pitchnames))

network_in = []
network_out = []

for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length]
    seq_out = notes[i+sequence_length]
    network_in.append([ele_to_int[ch] for ch in seq_in])
    network_out.append(ele_to_int[seq_out])    

In [20]:
len(network_in[0]),network_out[0]

(100, 297)

In [21]:
# No. of examples
n_patterns = len(network_in)
print(n_patterns)

60398


In [22]:
#1 is because LSTM accepsts data in 3D
network_in = np.reshape(network_in,(n_patterns,sequence_length,1))
print(network_in.shape)

(60398, 100, 1)


In [0]:
# Normalising Data

normalised_network_in = network_in/float(n_vocab)

In [0]:
#Network outputs are classes, encode into one hot vector

network_out = np_utils.to_categorical(network_out)

In [25]:
print(network_in.shape)
print(network_out.shape)

(60398, 100, 1)
(60398, 359)


## Create Model

In [0]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Sequential,load_model
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping

In [27]:
model = Sequential()
model.add(LSTM(512,input_shape=(100,1),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=False))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')
model.summary()



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________

In [0]:
#checkpoint = ModelCheckpoint("model.hdf5",monitor='loss',verbose=0,save_best_only=True,mode='min')

#model_his = model.fit(normalised_network_in,network_out,epochs=100,batch_size=64,callbacks=[checkpoint])

In [0]:
model.load_weights("/content/drive/My Drive/Colab Notebooks/midi_songs/new_weights.hdf5")

## Predictions

In [0]:
sequence_length = 100
network_in = []


for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length]
    network_in.append([ele_to_int[ch] for ch in seq_in])

In [40]:
start = np.random.randint(len(network_in)-1)
start 

6562

In [0]:
network_in[start]

# Mapping int to element
int_to_ele = dict((num,ele) for num,ele in enumerate(pitchnames))

#initial pattern
pattern = network_in[start]
prediction_out = []

#generate 200 elements
for note_index in range(200):
    prediction_in = np.reshape(pattern,(1,len(pattern),1))
    prediction_in = prediction_in/float(n_vocab)

    prediction = model.predict(prediction_in,verbose=0)

    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_out.append(result)

    #now taking the next sequence
    pattern.append(idx)
    pattern = pattern[1:]


In [43]:
print(len(prediction_out))

200


## Create Midi File

In [0]:
 #Time
 offset = 0
 output_notes = []

 for pattern in prediction_out:

     #If pattern is a chord 
     if ('+'  in pattern ) or pattern.isdigit():
         notes_in_chord = pattern.split('+')
         temp_notes = []
         for current_note in notes_in_chord:
             new_note = note.Note(int(current_note)) #create note object for each note in the chord
             new_note.storedInstrument = instrument.Piano()
             temp_notes.append(new_note)

         new_chord = chord.Chord(temp_notes) #creates chords from list of nodes
         new_chord.offset = offset    
         output_notes.append(new_note)
     
     else:
     #If pattern is a note
     
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

     offset += 0.5

In [0]:
output_notes

In [53]:
output_notes

[<music21.note.Note G#>]

In [63]:
# create a stream object from the generated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi',fp="test_output.mid")

'test_output.mid'